<a href="https://colab.research.google.com/github/fuushyn/DL2CR/blob/main/683_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# data_path = "trace.csv"

# df = pd.read_csv(data_path)


ips_test=["A","B","B","C","D","E","A","F","D","E","F","C"]
# [1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 0, 0]

cache_size = 2   #2MB cache

import numpy as np
from tqdm import tqdm

def opt_gen(ips, cache_size):
    occ_vec = np.zeros(len(ips), dtype=int)
    last_used = {}
    with tqdm(total=len(ips), desc="Processing IPs") as pbar:
        for i, ip in enumerate(ips):
            pbar.update(1)
            if ip in last_used:
                last_use = last_used[ip]
                max_occ = np.max(occ_vec[last_use:i])
                if max_occ >= cache_size:
                    continue
                occ_vec[last_use:i] += 1
            last_used[ip] = i
    return occ_vec


def get_labels(occ_vec):
    occ_vec = np.array(occ_vec)
    diff = np.diff(occ_vec)
    labels = np.where(diff > 0, 1, 0)
    return np.concatenate(([0], labels))


In [3]:
ov = opt_gen(ips_test, cache_size)
print(ov)
print(get_labels(ov))

Processing IPs: 100%|██████████| 12/12 [00:00<00:00, 3226.80it/s]

[1 2 1 1 2 2 1 2 1 1 0 0]
[0 1 0 0 1 0 0 1 0 0 0 0]


In [4]:
trace_path = '/content/drive/MyDrive/CS683_project/sssp-3.csv'
df = pd.read_csv(trace_path, dtype={'Instruction Pointer': str,'Source Memory 0': str, 'Destination Memory 0': str, }, header=0, skiprows=[1])
print(len(df["Instruction Pointer"]))
df = df[(df['Source Memory 0'] != '0') | (df['Destination Memory 0'] != '0')]
print(len(df["Instruction Pointer"]))

ips = df["Instruction Pointer"]
ov = opt_gen(ips, cache_size=2000)
labels = get_labels(ov)
df["Labels"] = labels
df.head()

4769373
1526590


Processing IPs: 100%|██████████| 1526590/1526590 [00:13<00:00, 113261.34it/s]


,Instruction Pointer,Is Branch,Branch Taken,Destination Register 0,Destination Register 1,Source Register 0,Source Register 1,Source Register 2,Source Register 3,Destination Memory 0,Destination Memory 1,Source Memory 0,Source Memory 1,Source Memory 2,Source Memory 3,Labels
0,559889caf3b0,1,1,26,0.0,26.0,0.0,0.0,0.0,0,0.0,559889ebedc0,0.0,0.0,0.0,0
1,14e493afd1d0,0,0,6,0.0,15.0,6.0,0.0,0.0,7ffeb7771480,0.0,0,0.0,0.0,0.0,0
2,14e493afd1d2,0,0,6,0.0,5.0,6.0,0.0,0.0,7ffeb7771478,0.0,0,0.0,0.0,0.0,0
4,14e493afd1d6,0,0,6,0.0,7.0,6.0,0.0,0.0,7ffeb7771470,0.0,0,0.0,0.0,0.0,0
8,14e493afd1e3,0,0,0,0.0,6.0,8.0,0.0,0.0,7ffeb77713d0,0.0,0,0.0,0.0,0.0,0


In [5]:
(df['Labels']>0).sum()

666

In [6]:
np.max(ov)

531

In [7]:
print(df["Instruction Pointer"].nunique())

1065


In [8]:
columns_to_keep = ['Instruction Pointer', 'Labels']
df = df[columns_to_keep]
df.head()


,Instruction Pointer,Labels
0,559889caf3b0,0
1,14e493afd1d0,0
2,14e493afd1d2,0
4,14e493afd1d6,0
8,14e493afd1e3,0


In [9]:
df.to_csv("cleaned_data_ssp-3.csv")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

# Load the dataset
# df = pd.read_csv('/mnt/data/cleaned_data_ssp-3.csv')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming the CSV has two columns: 'sequence' for the string and 'label' for the binary labels
sequences = df['Instruction Pointer'].values
labels = df['Labels'].values

# Encode the strings into integers
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(sequences)

# One-hot encode the integer encoded sequence
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# Pad sequences to the same length
max_sequence_length = 100  # or whatever is appropriate for your data
padded_sequences = pad_sequences(onehot_encoded, maxlen=max_sequence_length, padding='post')

# Convert to PyTorch tensors
padded_sequences = torch.tensor(padded_sequences, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Create TensorDatasets for training and validation sets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

# Create DataLoaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # Check if lstm_out has 3 dimensions, if not, it means it's only the last output
        if lstm_out.dim() == 3:
            # Use only the last output of the sequence for each batch
            logits = self.fc(lstm_out[:, -1, :])
        else:
            # Use the output as is since it's only the last time step's output
            logits = self.fc(lstm_out)
        return logits

# Set the number of epochs for training
num_epochs = 10000  # Define the number of epochs you wish to train for

# Initialize the model, loss function, and optimizer
model = LSTMModel(input_dim=max_sequence_length, hidden_dim=128).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Ensure GPU is available and move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for sequences, labels in train_loader:
        # Move tensors to the right device
        sequences = sequences.to(device)
        labels = labels.to(device).unsqueeze(1)  # Adjust labels' shape for BCEWithLogitsLoss

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    # Validation loop (optional, but recommended)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        valid_loss = 0
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device).unsqueeze(1)  # Adjust labels' shape for BCEWithLogitsLoss
            outputs = model(sequences)
            valid_loss += criterion(outputs, labels).item()
        valid_loss /= len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {valid_loss:.4f}')

# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/CS683_project/lstm_model.pth')


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch [1/10000], Loss: 0.0006, Validation Loss: 0.0036
Epoch [2/10000], Loss: 0.0002, Validation Loss: 0.0036
Epoch [3/10000], Loss: 0.0003, Validation Loss: 0.0036
Epoch [4/10000], Loss: 0.0006, Validation Loss: 0.0036
Epoch [5/10000], Loss: 0.0005, Validation Loss: 0.0036
Epoch [6/10000], Loss: 0.0004, Validation Loss: 0.0036
Epoch [7/10000], Loss: 0.0004, Validation Loss: 0.0036
Epoch [8/10000], Loss: 0.0005, Validation Loss: 0.0036
Epoch [9/10000], Loss: 0.0005, Validation Loss: 0.0036
Epoch [10/10000], Loss: 0.0004, Validation Loss: 0.0035
Epoch [11/10000], Loss: 0.0004, Validation Loss: 0.0036
Epoch [12/10000], Loss: 0.3307, Validation Loss: 0.0036
Epoch [13/10000], Loss: 0.0003, Validation Loss: 0.0036
Epoch [14/10000], Loss: 0.0006, Validation Loss: 0.0036
Epoch [15/10000], Loss: 0.0006, Validation Loss: 0.0036
Epoch [16/10000], Loss: 0.0005, Validation Loss: 0.0036
Epoch [17/10000], Loss: 0.0002, Validation Loss: 0.0036
Epoch [18/10000], Loss: 0.0004, Validation Loss: 0.0036
